In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import gdown
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import cv2
from tensorflow.keras.utils import Sequence
import albumentations as A
from albumentations.core.transforms_interface import DualTransform
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.regularizers import l2
from sklearn.utils.class_weight import compute_class_weight



print(tf.__version__)

In [ ]:
file_id = "1oQ6Vy_HqZlVHnkFspgxMn0IcE__D8Kmh"
zip_filename = "ocular-disease-recognition.zip"
extract_path = "./ocular-disease-recognition"

# Check if the file already exists
if not os.path.exists(zip_filename):
    print(f"Downloading {zip_filename}...")
    gdown.download(f"https://drive.google.com/uc?id={file_id}", zip_filename, quiet=False)
else:
    print(f"{zip_filename} already exists. Skipping download.")

In [ ]:
# Check if already extracted
if not os.path.exists(extract_path):
    os.makedirs(extract_path, exist_ok=True)
    print(f"Extracting {zip_filename}...")

    with zipfile.ZipFile(zip_filename, "r") as zip_ref:
        zip_ref.extractall(extract_path)

    print(f"Extraction complete! Files extracted to: {extract_path}")
else:
    print(f"Extraction skipped: {extract_path} already exists.")

In [ ]:
# Load the dataset (Update the path if necessary)
dataset_path = "processed_ocular_disease.csv"
df = pd.read_csv(dataset_path)

# **Split into Train, Validation, and Test Sets**
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)  # 70% Train, 30% Temp
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)  # 15% Val, 15% Test

In [ ]:
# **Get Number of Classes**
num_classes = len(np.unique(df['labels']))
print(f"Number of Classes: {num_classes}")

# Compute class weights
class_labels = np.unique(df['labels'])
class_weights = compute_class_weight(class_weight="balanced", classes=class_labels, y=df['labels'])
class_weight_dict = {i: class_weights[i] for i in range(len(class_labels))}

print("Computed Class Weights:", class_weight_dict)


In [ ]:
from sklearn.utils import resample

def balance_classes(df):
    """Resample dataset to balance classes."""
    max_size = df['labels'].value_counts().max()  # Find the max number of samples in any class
    balanced_df = pd.concat([
        resample(df[df['labels'] == cls], replace=True, n_samples=max_size, random_state=42)
        for cls in df['labels'].unique()
    ])
    return balanced_df.sample(frac=1).reset_index(drop=True)  # Shuffle after resampling

# Apply to training data only
train_df_balanced = balance_classes(train_df)

# Check if balancing worked
print(train_df_balanced['labels'].value_counts())  # Should now be balanced


In [ ]:
class DualImageAugmentation(DualTransform):
    def __init__(self, transforms, always_apply=False, p=0.5):
        super(DualImageAugmentation, self).__init__(always_apply, p)
        self.transforms = A.Compose(transforms)

    def apply(self, img, **params):
        return self.transforms(image=img)["image"]

    def apply_to_image1(self, img, **params):
        return self.transforms(image=img)["image"]

class OcularDatasetGenerator(Sequence):
    def __init__(self, df, batch_size=32, img_size=(128, 128), shuffle=True, augment=True, **kwargs):
        super().__init__(**kwargs)

        self.df = df
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.augment = augment  
        self.indices = np.arange(len(df))
        
        # Define augmentation pipeline if augmentation is enabled
        if augment:
            self.augmentation_pipeline = self.get_augmentation_pipeline()
        else:
            self.augmentation_pipeline = None

        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))  

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch = self.df.iloc[batch_indices]
        X, y = self.__data_generation(batch)
        return np.array(X), np.array(y)  

    def __data_generation(self, batch):
        X_batch = []
        y_batch = []
        
        for _, row in batch.iterrows():
            left_image_path = os.path.join('ocular-disease-recognition/preprocessed_images', row['Left-Fundus'])
            right_image_path = os.path.join('ocular-disease-recognition/preprocessed_images', row['Right-Fundus'])
            
            left_image = self.load_image(left_image_path)
            right_image = self.load_image(right_image_path)

            if left_image is None or right_image is None:
                continue

            # Apply identical augmentations to both images
            if self.augment and self.augmentation_pipeline:
                augmented = self.augmentation_pipeline(image=left_image, image1=right_image)
                left_image = augmented["image"]
                right_image = augmented["image1"]

            # Ensure channel dimension is included (for grayscale images)
            left_image = np.expand_dims(left_image, axis=-1)  # Shape: (128, 128, 1)
            right_image = np.expand_dims(right_image, axis=-1)  # Shape: (128, 128, 1)

            combined_image = np.concatenate((left_image, right_image), axis=-1)  # Shape: (128, 128, 2)
            X_batch.append(combined_image)

            y_batch.append(int(row['labels']))  

        return np.array(X_batch, dtype=np.float32), np.array(y_batch, dtype=np.int32)

    def load_image(self, image_path):
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            return None
        image = cv2.resize(image, self.img_size)
        image = image / 255.0  
        return image

    def get_augmentation_pipeline(self):
        return DualImageAugmentation([
            A.RandomBrightnessContrast(p=0.4),
            A.GaussianBlur(blur_limit=(3, 7), p=0.3), 
            A.HorizontalFlip(p=0.5),
            A.Affine(rotate=(-20, 20), scale=(0.9, 1.1), translate_percent=(0.05, 0.05), p=0.6), 
            A.ElasticTransform(p=0.3),
            A.GridDistortion(p=0.3),
            A.RandomRotate90(p=0.3),
            A.CLAHE(p=0.2)  # Contrast enhancement
        ])
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

In [ ]:
# **Create Data Generators**
# batch_size = 32
batch_size = 16
train_generator = OcularDatasetGenerator(train_df_balanced, batch_size=batch_size, img_size=(224, 224), augment=True)
val_generator = OcularDatasetGenerator(val_df, batch_size=batch_size, img_size=(224, 224))
test_generator = OcularDatasetGenerator(test_df, batch_size=batch_size, shuffle=False, img_size=(224, 224))


# **Define CNN Model** 
model = models.Sequential([
    layers.Input(shape=(224, 224, 2)),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),

    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),

    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.4),

    layers.Conv2D(512, (3, 3), activation='relu', padding='same'), 
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.4),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),

    layers.Dense(num_classes, activation='softmax')
])

def step_decay(epoch):
    initial_lr = 0.001
    drop = 0.5
    epochs_drop = 3
    return initial_lr * (drop ** (epoch // epochs_drop))

lr_schedule = tf.keras.callbacks.LearningRateScheduler(step_decay)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Start with 0.001
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


# **Define Early Stopping Callback**
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',    # Stop if validation loss stops improving
    patience=5,            # Wait for 5 epochs before stopping
    restore_best_weights=True  # Restore best model weights
)

# **Train the Model with Early Stopping**
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=val_generator,
    callbacks=[early_stopping, lr_schedule],
)

In [ ]:
# **Evaluate the Model on Test Set**
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")